# Jeux avec adversaires

Dans ce TP, nous allons implémenter des jeux avec adversaires, en prenant le jeu du tic-tac-toe et celui du Puissance 4.

In [495]:
from collections import defaultdict
import random
import math
from typing import List, Any

## Classe `Game`

On définit tout d'abord une classe abstraite `Game`, pour représenter des jeux avec adversaires qui jouent à tour de rôle.

Cette représentation s'appuie sur le concept d'état, `state`, qui sera défini par chacun des jeux à proprement parlé. Chaque état requiert la définition d'un attribut `state.to_move`, qui donne le nom du joueur dont c'est le tour de jeu (par exemple, pour le tic-tac-toe, un joueur sera `'X'` et l'autre joueur sera `'O'`). 


In [496]:
class State:
    def __init__(self, to_move: str):
        self.to_move = to_move

In [497]:
class Game:
    """A game is similar to a problem, but it has a terminal test instead of 
    a goal test, and a utility for each terminal state. To create a game, 
    subclass this class and implement `actions`, `result`, `is_terminal`, 
    and `utility`. You will also need to set the .initial attribute to the 
    initial state; this can be done in the constructor."""

    def actions(self, state: State):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state: State, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state: State):
        """Return True if this is a final state for the game."""
        return not self.actions(state)
    
    def utility(self, state: State, player: str):
        """Return the value of this final state to player."""
        raise NotImplementedError
    
    def process_human_input(self, input: str):
        """Convert human input into a game state."""
        raise NotImplementedError
        


## Algorithme de jeu min-max

Chaque algorithme de jeu prend 2 paramètres : le jeu considéré (représenté par un objet de type `Game`) et l'état courant du jeu (représenté par un objet de type `State`). Cet algorithme retourne une paire `(value, move)` dans laquelle `value` est la valeur de fonction d'évaluation donnée par l'algorithme pour le joueur dont c'est le tour de jeu et `move` est le déplacement à effectuer.


In [498]:
# constante pour représenter la valeur infinie (peut ensuite être utilisée en +infinity ou -infinity)
infinity = math.inf

### Question 1 : implémenter la méthode `minimax_search`, en implémentant les 2 sous-fonctions `max_value` et `min_value` (voir la définition de ces fonctions donnée sur le slide 11 du cours).

In [499]:
"""Search game tree to determine best move; return (value, move) pair."""
def minimax_search(game: Game, state: State) -> tuple[float, Any]:
    player = state.to_move

    def max_value(state: State) -> tuple[float, Any]:
        if game.is_terminal(state):
            return game.utility(state, player), state
        
        best_value = -infinity
        best_action = None

        for action in game.actions(state):
            new_value, _ = min_value(game.result(state, action))

            if new_value > best_value:
                best_value = new_value
                best_action = action

        return best_value, best_action


    def min_value(state: State) -> tuple[float, Any]:
        if game.is_terminal(state):
            return game.utility(state, player), state
        
        best_value = +infinity
        best_action = None

        for action in game.actions(state):
            new_value, _ = max_value(game.result(state, action))

            if new_value < best_value:
                best_value = new_value
                best_action = action

        return best_value, best_action

    return max_value(state)


## Classe `Board`

Les états du tic-tac-toe sont représentés par un plateau de jeu, `Board`, qui est une sous-classe de `defaultdict`? Cette dernière consiste en un dictionnaire de couples `{(x, y): contents}`. Par exemple, `{(0, 0): 'X', (1, 1): 'O'}` sera l'état du plateau de jeu, obtenu après 2 tours de jeu. En plus du contenu des cases, un plateau de jeu a d'autres attributs : 
- `.to_move` pour indiquer le nom du joueur dont c'est le tour de jeu ; 
- `.width` et `.height` pour donner la largeur et la hauteur du plateau de jeu (toutes les 2 égales à 3 pour le tic-tac-toe mais avec des valeurs différentes pour d'autres jeux tels que le Puissance 4) ;
- possiblement d'autres attributs, spécifiés par des mots-clés. 

En tant qu'objet `defaultdict`, la classe `Board` a une méthode `__missing__`, qui retourne `empty` pour les cases qui n'ont pas encore été jouées mais qui sont comprises dans le plateau de jeu (à l'intérieur des frontières du plateau défini par `width` × `height`) et `off` sinon (pour les cases hors des frontières du plateau de jeu). 
La classe a une méthode `__hash__` pour que les objets puissent être stockés dans des tables de hachage.

In [500]:
class Board(defaultdict, State):
    """A board has the player to move, a cached utility value, 
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""
    empty = '.'
    off = '#'
    
    def __init__(self, width=8, height=8, to_move=None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)
        
    def new(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self, loc: tuple[int, int]):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)
    
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'

## Joueurs

Nous avons besoin d'une interface pour représenter des joueurs.
Un joueur sera représenté par une fonction qui aura 2 paramètres, `(game, state)`, et qui retournera un coup, `move`.



La fonction `player` crée un joueur en spécifiant l'algorithme de jeu utilisé.

In [501]:
def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

La fonction `random_player` représente un joueur qui choisit un coup à jouer, de manière aléatoire.

In [502]:
def random_player(game, state): 
    return random.choice(list(game.actions(state)))

### Question 2 : implémenter la fonction human_player qui demande d'entrer un coup à jouer (au clavier) et qui retourne le coup choisi.

In [503]:
from time import sleep

def human_player(game: Game, state: State):
    print(state)
    sleep(2)
    action_str = input("Select your next move position:")
    return game.process_human_input(action_str)

## Jouer une partie d'un jeu avec adversaire

Pour pouvoir jouer une partie avec 2 joueurs, on définit tout d'abord la fonction `play_game` qui prend, en paramètres, un objet `Game` et un dictionnaire de 2 éléments `{player_name: strategy_function}` (on associe ainsi une stratégie de jeu à chaque joueur dont on donne le nom) et qui joue un jeu avec adversaire. A chaque tour de jeu, on utilise `state.to_move` pour savoir quel joueur doit jouer le tour de jeu courant et on utilise la stratégie de jeu associée au joueur courant pour choisir le coup à jouer.

In [504]:
"""Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
def play_game(game: Game, strategies: dict, verbose=False):
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose: 
            print('Player', player, 'move:', move)
            print(state)
    return state

## Jeu 1 : jeu du Tic-Tac-Toe

Les coups sont définis par des paires `(x, y)`, avec `(0, 0)` le coup en haut à gauche et `(2, 2)` celui en bas à droite (le plateau de jeu est de taille `height=width=3`).

### Question 3 : implémenter la classe `TicTacToe`.

In [505]:
class TicTacToe(Game):
    """Play TicTacToe on an `height` by `width` board, needing `k` in a row to win.
    'X' plays first against 'O'."""

    player_x = 'X'
    player_o = 'O'

    def __init__(self, height=3, width=3, k=3):
        self.k = k # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move=self.player_x, utility=0)

    def actions(self, board: Board) -> List[tuple[int, int]]:
        possible = []

        for square in self.squares:
            if board[square] == board.empty:
                possible.append(square)

        return possible

    def result(self, board: Board, square: tuple[int, int]) -> Board:
        """Place a marker for current player on square."""
        new_board = board.new({square: board.to_move})

        if board.to_move == self.player_x:
            new_board.to_move = self.player_o
        else:
            new_board.to_move = self.player_x

        return new_board

    def utility(self, board: Board, player: str) -> int:
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        if player == self.player_x:
            mult = 1
        else:
            mult = -1

        for square in self.squares:
            if k_in_row(board, self.player_x, square, self.k):
                return mult*1
            
            if k_in_row(board, self.player_o, square, self.k):
                return mult*-1
        
        return 0

    def is_terminal(self, board: Board) -> bool:
        """A board is a terminal state if it is won or there are no empty squares."""
        no_empty = True

        for square in self.squares:
            if no_empty:
                if board[square] == board.empty:
                    no_empty = False

            if k_in_row(board, self.player_x, square, self.k):
                return True
            
            if k_in_row(board, self.player_o, square, self.k):
                return True

        return no_empty

    def display(self, board): 
        print(board)  

    def process_human_input(self, input: str) -> tuple[int, int]:
        pos = input.split(",")

        return int(pos[0]), int(pos[1])


def k_in_row(board, player, square, k):
    """True if player has k pieces in a line through square."""
    def in_row(x, y, dx, dy): 
        return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

### Jouer une partie de tic-tac-toe

On peut maintenant jouer une partie entre 2 joueurs, choisis parmi 3 types de joueurs : `random_player`, `human_player` et `player` (en choisissant une stratégie de jeu, ici min-max).

#### Partie de Tic-Tac-Toe entre 2 joueurs aléatoires


In [506]:
# partie de Tic-Tac-Toe entre 2 joueurs aléatoires
print(play_game(TicTacToe(), dict(X=random_player, O=random_player), verbose=True))

Player X move: (0, 0)
X . .
. . .
. . .

Player O move: (1, 2)
X . .
. . .
. O .

Player X move: (0, 2)
X . .
. . .
X O .

Player O move: (1, 1)
X . .
. O .
X O .

Player X move: (2, 2)
X . .
. O .
X O X

Player O move: (0, 1)
X . .
O O .
X O X

Player X move: (2, 0)
X . X
O O .
X O X

Player O move: (2, 1)
X . X
O O O
X O X

X . X
O O O
X O X



#### Question 4 : reprenez l'appel à la fonction `play_game` pour faire jouer des joueurs avec un stratégie de jeu min-max ainsi qu'un joueur humain.

#### Partie de jeu entre un joueur aléatoire et un joueur min-max

In [507]:
print(play_game(TicTacToe(), dict(X=random_player, O=player(minimax_search)), verbose=True))

Player X move: (2, 2)
. . .
. . .
. . X



Player O move: (1, 1)
. . .
. O .
. . X

Player X move: (0, 2)
. . .
. O .
X . X

Player O move: (1, 2)
. . .
. O .
X O X

Player X move: (0, 0)
X . .
. O .
X O X

Player O move: (0, 1)
X . .
O O .
X O X

Player X move: (2, 0)
X . X
O O .
X O X

Player O move: (2, 1)
X . X
O O O
X O X

X . X
O O O
X O X



#### Partie de jeu entre un joueur aléatoire et un joueur humain

In [511]:
print(play_game(TicTacToe(), dict(X=player(minimax_search), O=human_player)))

. . .
X . .
. . .

. . .
X O .
. X .

X . .
X O .
. X O

X . X
X O .
O X O

X O X
X O X
O X O



## Jeu 2 : Puissance 4

Le Puissance 4 est une variante du tic-tac-toe, joué sur un plateau plus grand (de taille 7x6), et avec la restriction de pouvoir jouer dans un colonne uniquement dans la position la plus basse de celle-ci. 
Seuls le constructeur et la fonction `actions` sont à redéfinir, par rapport à la classe `TicTacToe`.

### Question 5 : compléter la classe `ConnectFour`.

In [ ]:
class ConnectFour(TicTacToe):
    
    def __init__(self): super().__init__(width=7, height=6, k=4)

    def actions(self, board):
        """In each column you can play only the lowest empty square in the column."""
        raise NotImplementedError

### Jouer une partie de Puissance 4

On peut maintenant jouer une partie entre 2 joueurs, choisis parmi 3 types de joueurs : `random_player`, `human_player` et `player`.

In [ ]:
play_game(ConnectFour(), dict(X=random_player, O=random_player), verbose=True).utility

NotImplementedError: 

### Question 6 : reprenez l'appel à la fonction `play_game` pour faire jouer des joueurs avec un stratégie de jeu min-max ainsi qu'un joueur humain.

In [ ]:
#todo